In [16]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd

Must remove before submission, just here becasue .env issues. 

In [17]:
os.environ["NOAA_WEB_API_TOKEN"] = "zBZyicEUuGGoKumtcmeBWTCGirGjQuCy"

NOAA_WEB_API_TOKEN = os.getenv("NOAA_WEB_API_TOKEN")
print(NOAA_WEB_API_TOKEN)

zBZyicEUuGGoKumtcmeBWTCGirGjQuCy


## Acquiring 2025 temperatures for Washington, DC
I used the Global Historical Climatology Network (GHCN), avaiable throughthe NOAA web API, to acquire these temperatures. 

In [18]:
# load_dotenv(dotenv_path="NOAA_WEB_API_TOKEN.env")
NOAA_WEB_API_TOKEN = os.getenv("NOAA_WEB_API_TOKEN")
assert NOAA_WEB_API_TOKEN is not None, "Token not found! Check your environment variable."

NOAA_API_BASE_URL = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"

# Station IDs for the specified locations
stations  = {
  "washingtondc": "GHCND:USW00013743",
  "vancouver": "GHCND:CA001108395",
  "newyorkcity": "GHCND:USW00014732",
  "liestal": "GHCND:SZ000001940",
  "kyoto": "GHCND:JA000047759"
}

headers = {
    "token": NOAA_WEB_API_TOKEN
}

params = {
    "datasetid": "GHCND",
    "stationid": "GHCND:USW00013743",
    "startdate": "2025-01-01",
    "enddate": "2025-12-31",
    "datatypeid": ["TMIN", "TMAX", "TAVG"],
    "units": "standard",
    "limit": 1000
}

response = requests.get(
    "https://www.ncei.noaa.gov/cdo-web/api/v2/data",
    headers=headers,
    params=params
)

print(response.status_code)
data = response.json()
#print(data)

if "results" in data:
    df = pd.DataFrame(data["results"])
else:
    print("No results returned:", data)

200


### Cleaning JSON 

In [19]:
df['date'] = pd.to_datetime(df['date'])

# Pivot to wide format
df_wide = df.pivot(index='date', columns='datatype', values='value').reset_index()

df_wide.columns.name = None
df_wide = df_wide.rename(columns={
    'TMIN': 'tmin',
    'TMAX': 'tmax',
    'TAVG': 'tavg'
})

# If TAVG is missing, computed as the mean of min/max
if df_wide['tavg'].isna().any():
    df_wide['tavg'] = df_wide[['tmin','tmax']].mean(axis=1)
print(df_wide.shape)
print(df_wide.isna().sum())

(335, 4)
date    0
tavg    0
tmax    0
tmin    1
dtype: int64


API had some missing values that were left out, so I am combining the results of the API with a dataset that is the full length of the year. 

In [20]:
# Full 2025 date range
full_2025 = pd.DataFrame({'date': pd.date_range(start='2025-01-01', end='2025-12-31')})


dc_2025_full = full_2025.merge(df_wide, on='date', how='left')

# Fill missing TMIN/TMAX with interpolation
dc_2025_full['tmin'] = dc_2025_full['tmin'].interpolate()
dc_2025_full['tmax'] = dc_2025_full['tmax'].interpolate()
dc_2025_full['tavg'] = dc_2025_full['tavg'].interpolate()

print(dc_2025_full.isna().sum()) 
print(dc_2025_full.shape)  

date    0
tavg    0
tmax    0
tmin    0
dtype: int64
(365, 4)


In [21]:
dc_2025_full.to_csv("Data/dc_Temps_2025.csv", index=False)

### Combining Temperature data sets

In [22]:
temps_dc = pd.read_csv('Q_temps_dc.csv')
full_temps_dc = pd.concat([temps_dc, dc_2025_full], ignore_index=True)

full_temps_dc['date'] = pd.to_datetime(full_temps_dc['date'])

full_temps_dc['doy'] = full_temps_dc['date'].dt.dayofyear

full_temps_dc['year'] = full_temps_dc['date'].dt.year

if 'temp' in full_temps_dc.columns:
    full_temps_dc = full_temps_dc.drop(columns=['temp'])

full_temps_dc = full_temps_dc[['year','date', 'doy', 'tmin','tmax','tavg']]

print(full_temps_dc.tail())
print(full_temps_dc.isna().sum())

full_temps_dc.to_csv('Data/full_temps_dc.csv', index=False)

       year       date  doy  tmin  tmax  tavg
30638  2025 2025-12-27  361  37.0  46.0  46.0
30639  2025 2025-12-28  362  37.0  46.0  46.0
30640  2025 2025-12-29  363  37.0  46.0  46.0
30641  2025 2025-12-30  364  37.0  46.0  46.0
30642  2025 2025-12-31  365  37.0  46.0  46.0
year        0
date        0
doy         0
tmin        0
tmax        0
tavg    30278
dtype: int64


### Adding 2025 bloom date to bloom data set

In [23]:
cherry_dc = pd.read_csv('Q_blooms_dc.csv')
print(cherry_dc.tail())

    year  bloom_date  bloom_doy
78  2020  2020-03-20         80
79  2021  2021-03-28         87
80  2022  2022-03-21         80
81  2023  2023-03-23         82
82  2024  2024-03-17         77


In [24]:
bloom_date_2025 = [2025, '2025-03-28', 87]
cherry_dc.loc[len(df)] = bloom_date_2025

cherry_dc['bloom_date'] = pd.to_datetime(cherry_dc['bloom_date'])

print(cherry_dc.tail())

      year bloom_date  bloom_doy
79    2021 2021-03-28         87
80    2022 2022-03-21         80
81    2023 2023-03-23         82
82    2024 2024-03-17         77
1000  2025 2025-03-28         87
